## Construct Health Expenditure Performance Index and Health Expenditure Efficiency Index

### 📝 Note: Health Expenditure Performance Index (HEP)

**HEE = Health Expenditure Efficiency Index (HEE)

Clean the following data (for missing data)
Life expectancy (years), Infant Mortality (Death rate per 1000 people), Average Schooling (years), Learning Outcome (scores), Health Expenditure (% of GDP), Education Expenditure (% of GDP)

Replicate PSP model by using the following data to generate 

HEP(Health Expenditure Performance) 
Independent variables = Life expectancy, Infant Mortality, Average Schooling (years), Learning Outcome

After that we will generate HEE (Health Expenditure Efficiency) by divided the HEP by Expenditure


HEE = HEP/(Health Expenditure + Education Expenditure)

HEP = sum of Life expectancy, Infant Mortality, Average Schooling (years), Learning Outcome (Scores)
→ Methodology, normalized each independent variables before combine to HEP index, noted that Infant Mortality using inverse value since the less is better
→ Higher HEP means better standard of living

HEE = HEP/ (Health Expenditure + Education Expenditure) 
→ to evaluate the efficiency of government spending 
→ Higher HEE means using lower health and education expenditure but get better HEP
In order to compute efficiency indicators, government spending was normalized across countries, with the average taking the value of one for each of the two categories (Health Expenditure, Education Expenditure)

For each indicator (Health and Education Expenditure):

Normalize each country's value by dividing it by the cross-country average, so that:

normalized value = raw value/ mean value across countries

The result will have a mean of 1, but preserve relative differences.
(The reason we use normalized value instead of raw value is to prevent countries with large economies (e.g., USA, Germany) appear less efficient, simply because they spend more in absolute terms. Also, developing countries might look artificially efficient, even if their HEP is low, simply because their raw expenditure is very small. Therefore, using normalized value will remove scale bias)


In [ ]:
# === Step 0: Imports and setup ===
import pandas as pd
import os

# === Step 1: Load and clean merged data ===
df = pd.read_csv("../data/interim/merged_data.csv")
df = df[df["Year"] <= 2022]
df = df.sort_values(by=["Country", "Year"], ascending=[True, False])

# === Step 1.1: Forward-fill and backfill per country group (warning-free) ===
df_cleaned = (
    df.groupby("Country", group_keys=False)
      .apply(lambda g: g.bfill().ffill())
      .reset_index(drop=True)
)

# === Step 2: Normalize indicators and compute sub-indices ===
def normalize(s):
    return (s - s.min()) / (s.max() - s.min())

df_norm = df_cleaned.copy()
df_norm["life_expectancy_norm"]    = normalize(df_norm["Life_Expectancy"])
df_norm["infant_mortality_norm"]  = 1 - normalize(df_norm["Mortality_Rate"])
df_norm["average_schooling_norm"] = normalize(df_norm["average_schooling"])
df_norm["learning_outcome_norm"]  = normalize(df_norm["learning_scores"])

df_norm["health_index_raw"]      = df_norm[["life_expectancy_norm", "infant_mortality_norm"]].mean(axis=1)
df_norm["education_index_raw"]   = df_norm[["average_schooling_norm", "learning_outcome_norm"]].mean(axis=1)

df_norm["health_index"]     = df_norm["health_index_raw"]    / df_norm["health_index_raw"].mean()
df_norm["education_index"]  = df_norm["education_index_raw"] / df_norm["education_index_raw"].mean()

# === Step 3: Calculate HEP and normalize expenditures ===
df_norm["HEP"] = df_norm[["health_index", "education_index"]].mean(axis=1)

df_norm["Health_Expenditure_norm"] = df_norm["Health_Expenditure"] / df_norm["Health_Expenditure"].mean()
df_norm["Education_Expenditure_norm"] = df_norm["Education_Expenditure"] / df_norm["Education_Expenditure"].mean()

# === Step 4: Calculate HEE and sub-components ===
df_norm["health_eff_index"] = df_norm["health_index"] / df_norm["Health_Expenditure_norm"]
df_norm["education_eff_index"] = df_norm["education_index"] / df_norm["Education_Expenditure_norm"]

df_norm["HEE"] = df_norm[["health_eff_index", "education_eff_index"]].mean(axis=1)

# For consistency with earlier versions, also save individual subcomponents
df_norm["HEP_Health"] = df_norm["health_index"]
df_norm["HEP_Edu"]    = df_norm["education_index"]
df_norm["HEE_Health"] = df_norm["health_eff_index"]
df_norm["HEE_Edu"]    = df_norm["education_eff_index"]

# === Step 5: Save final result ===
result = df_norm[[ 
    "ISO3", "Year", "income_level",
    "HEP", "HEE",
    "HEP_Health", "HEE_Health",
    "HEP_Edu", "HEE_Edu"
]]

os.makedirs("../data/processed", exist_ok=True)
result.to_csv("../data/processed/hep_hee_results.csv", index=False)

print("✅ HEP & HEE (+ health/edu split) calculated and saved successfully.")
print(result.tail())


✅ HEP & HEE (+ health/edu split) calculated and saved successfully.
     ISO3  Year         income_level       HEP       HEE  HEP_Health  \
1260  VNM  2004  Lower middle income  1.031994  1.231556    0.992324   
1261  VNM  2003  Lower middle income  1.024156  1.253471    0.991247   
1262  VNM  2002  Lower middle income  1.013597  1.256908    0.984727   
1263  VNM  2001  Lower middle income  1.002204  1.086351    0.976540   
1264  VNM  2000  Lower middle income  0.989390  1.182313    0.965511   

      HEE_Health   HEP_Edu   HEE_Edu  
1260    1.565361  1.071664  0.897750  
1261    1.621421  1.057065  0.885520  
1262    1.640526  1.042467  0.873291  
1263    1.311641  1.027868  0.861061  
1264    1.515794  1.013269  0.848831  


/var/folders/py/4z0fk4cn3_dcjpjscqzv_9jr0000gn/T/ipykernel_33386/3092926774.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("Country", group_keys=False)


Add region and average index score over years for each country

In [2]:
import pandas as pd
import os

# === Auto-install pycountry if missing ===
try:
    import pycountry
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pycountry"])
    import pycountry

# === Step 1: Load the data ===
data_path = "../data/processed/hep_hee_results.csv"
df = pd.read_csv(data_path)

# === Step 2: Define compact country-to-region mapping ===
emea = [
    "ARM", "CYP", "CZE", "EGY", "ETH", "DEU", "FRA", "GRC", "IRL", "JOR", "KAZ", "KEN", "KGZ",
    "LBN", "MAR", "NLD", "NGA", "ROU", "SRB", "SVK", "TJK", "TUN", "TUR", "UKR", "GBR", "UZB", "RUS", "IRN"
]
apac = [
    "AUS", "BGD", "CHN", "IND", "IDN", "JPN", "MYS", "MNG", "MMR",
    "NZL", "PAK", "PHL", "SGP", "THA", "VNM", "KOR"
]
latam = [
    "ARG", "BRA", "CHL", "ECU", "GTM", "MEX", "NIC", "PER", "URY"
]
na = ["CAN", "USA"]

# Combine into a single dictionary
country_region_map = {
    **{ISO3: "EMEA" for ISO3 in emea},
    **{ISO3: "APAC" for ISO3 in apac},
    **{ISO3: "LATAM" for ISO3 in latam},
    **{ISO3: "NA" for ISO3 in na},
}

# === Step 3: Map region and country name ===
df["Region"] = df["ISO3"].map(country_region_map)

def get_country_name(iso3):
    try:
        return pycountry.countries.get(alpha_3=iso3).name
    except:
        return "Unknown"

df["Country"] = df["ISO3"].apply(get_country_name)

# === Step 4: Warn unmapped ISO3 codes ===
unmapped = df[df["Region"].isnull()]["ISO3"].unique()
if len(unmapped) > 0:
    print("⚠️ Warning: The following countries couldn't be mapped to a region:")
    for c in unmapped:
        print(f"- {c}")
    print("👉 Please update 'country_region_map' with these countries.")

# === Step 5: Save updated file ===
output_path = "../data/processed/hep_hee_results_with_region.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)
print(f"✅ 'Region' and 'Country' columns added and saved to: {output_path}")


✅ 'Region' and 'Country' columns added and saved to: ../data/processed/hep_hee_results_with_region.csv


Visualize HEE and HEP index

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output

# === Step 1: Load and prepare data ===
df = pd.read_csv("../data/processed/hep_hee_results_with_region.csv")
df = df[df["income_level"].notnull()]
df["Year"] = df["Year"].astype(int)
df = df.sort_values(by="Year")
df["Year_str"] = df["Year"].astype(str)

# === Step 2: Compute average values ===
df_avg = df[df["Year"].between(2000, 2022)]
df_avg = df_avg.groupby(["ISO3", "income_level"])[["HEP", "HEE", "HEP_Health", "HEE_Health", "HEP_Edu", "HEE_Edu"]].mean().reset_index()
df_avg["text"] = df_avg["ISO3"] + " (avg)"

# === Step 3: Get options ===
available_countries = sorted(df["ISO3"].unique())
country_options = [{"label": "All Countries", "value": "ALL"}] + [
    {"label": iso3, "value": iso3} for iso3 in available_countries
]
available_income_levels = sorted(df["income_level"].dropna().unique())
income_options = [{"label": level, "value": level} for level in available_income_levels]
avg_toggle = [{"label": "Show 2000–2022 Average", "value": "show_avg"}]

# === Step 4: Build Dash App ===
app = dash.Dash(__name__)
app.title = "HEP vs HEE Scatter and Ranking"

app.layout = html.Div([
    html.H2("HEP vs HEE – Animated and Average Scatter by Country"),

    html.Div([
        html.Label("Select Countries (ISO3):"),
        dcc.Dropdown(id="country-dropdown", options=country_options, value=["ALL"], multi=True),
    ], style={"width": "60%", "marginBottom": "20px"}),

    html.Div([
        html.Label("Select Income Levels:"),
        dcc.Dropdown(id="income-dropdown", options=income_options, value=available_income_levels, multi=True),
    ], style={"width": "60%", "marginBottom": "20px"}),

    html.Div([
        dcc.Checklist(id="avg-toggle", options=avg_toggle, value=["show_avg"],
                      labelStyle={"display": "inline-block", "marginRight": "10px"})
    ], style={"marginBottom": "20px"}),

    dcc.Graph(id="animated-scatter-main", style={"height": "600px"}),
    dcc.Graph(id="animated-scatter-health", style={"height": "600px"}),
    dcc.Graph(id="animated-scatter-edu", style={"height": "600px"}),

    html.Hr(),
    html.H3("HEP vs HEE (2000–2022 Avg Only, No Animation)"),

    dcc.Graph(id="avg-scatter-main"),
    dcc.Graph(id="avg-scatter-health"),
    dcc.Graph(id="avg-scatter-edu"),

    html.Hr(),
    html.H3("Avg HEP vs HEE Bar Chart (2000–2022)"),
    dcc.Graph(id="bar-chart-hep-hee"),
])

# === Step 5: Callback ===
@app.callback(
    Output("animated-scatter-main", "figure"),
    Output("animated-scatter-health", "figure"),
    Output("animated-scatter-edu", "figure"),
    Output("avg-scatter-main", "figure"),
    Output("avg-scatter-health", "figure"),
    Output("avg-scatter-edu", "figure"),
    Output("bar-chart-hep-hee", "figure"),
    Input("country-dropdown", "value"),
    Input("income-dropdown", "value"),
    Input("avg-toggle", "value")
)
def update_figure(selected_countries, selected_income_levels, avg_toggle):
    if "ALL" in selected_countries:
        selected_countries = available_countries

    dff = df[df["ISO3"].isin(selected_countries) & df["income_level"].isin(selected_income_levels)]
    dff_avg = df_avg[df_avg["ISO3"].isin(selected_countries) & df_avg["income_level"].isin(selected_income_levels)]

    def make_animated_scatter(x_col, y_col, title, avg_x, avg_y):
        fig = px.scatter(
            dff, x=x_col, y=y_col, color="income_level", text="ISO3",
            animation_frame="Year_str", range_x=[0, 1], range_y=[0, 1],
            labels={x_col: f"Performance ({x_col})", y_col: f"Efficiency ({y_col})"},
            title=title, width=950, height=600
        )
        fig.update_traces(marker=dict(size=10), textposition="top center")

        if "show_avg" in avg_toggle:
            fig.add_trace(go.Scatter(
                x=dff_avg[avg_x], y=dff_avg[avg_y], mode="markers+text",
                text=dff_avg["text"], textposition="bottom center",
                marker=dict(size=9, symbol="diamond", color="black", opacity=0.4),
                name="2000–2022 Avg"
            ))

        fig.add_shape(
            type="line", x0=0, y0=0, x1=1, y1=1,
            xref="x", yref="y", line=dict(dash="dash", color="gray")
        )
        return fig

    def make_static_avg_scatter(x_col, y_col, title):
        fig = px.scatter(
            dff_avg, x=x_col, y=y_col, color="income_level", text="ISO3",
            range_x=[0, 1.5], range_y=[0, 2.5],
            labels={x_col: f"Performance ({x_col})", y_col: f"Efficiency ({y_col})"},
            title="", width=950, height=600
        )
        fig.update_traces(marker=dict(size=10), textposition="top center")
        #fig.add_shape(
            #type="line", x0=0, y0=0, x1=1, y1=1,
            #xref="x", yref="y", line=dict(dash="dash", color="gray")
        #)
        return fig

    def make_bar_chart():
        # 1) sort ascending so smallest HEP is at the bottom
        df_sorted = dff_avg.sort_values("HEP", ascending=True)

        fig = go.Figure()

        # 2) note x=values, y=categories, orientation='h'
        fig.add_trace(go.Bar(
        x=df_sorted["HEP"],
        y=df_sorted["ISO3"],
        orientation='h',
        name="Avg HEP",
        marker_color="red"
        ))
        fig.add_trace(go.Bar(
        x=df_sorted["HEE"],
        y=df_sorted["ISO3"],
        orientation='h',
        name="Avg HEE",
        marker_color="orange"
        ))

        # 3) group them, reverse the y-axis so highest HEP appears at top
        fig.update_layout(
        barmode='group',
        title_text="Avg HEP vs HEE index by Country (2000–2022)",
        xaxis_title="Index Value",
        yaxis_title="Country",
        yaxis=dict(autorange="reversed"),
        height=1000,
        margin=dict(l=100, r=20, t=50, b=50)
        )
        
        return fig

    fig_main = make_animated_scatter("HEP", "HEE", "HEP vs HEE Index (Overall)", "HEP", "HEE")
    fig_health = make_animated_scatter("HEP_Health", "HEE_Health", "HEP vs HEE Index (Health Only)", "HEP_Health", "HEE_Health")
    fig_edu = make_animated_scatter("HEP_Edu", "HEE_Edu", "HEP vs HEE Index (Education Only)", "HEP_Edu", "HEE_Edu")

    fig_avg_main = make_static_avg_scatter("HEP", "HEE", "Avg HEP vs HEE Index (Overall)")
    fig_avg_health = make_static_avg_scatter("HEP_Health", "HEE_Health", "Avg HEP vs HEE Index (Health Only)")
    fig_avg_edu = make_static_avg_scatter("HEP_Edu", "HEE_Edu", "Avg HEP vs HEE Index (Education Only)")

    fig_bar = make_bar_chart()

    fig_avg_main.update_layout(margin=dict(l=20, r=20, t=20, b=20))

    return fig_main, fig_health, fig_edu, fig_avg_main, fig_avg_health, fig_avg_edu, fig_bar

# === Step 6: Run App ===
if __name__ == "__main__":
    app.run(debug=True)
